# Unsupervised learning Capstone (name TBA)
Author: Matthew Huh
    
## Overview

For the most part, people are free to choose what news outlets they read and follow. In the United States, there is a near-endless list of sites that people can choose from in order to get their daily news and over time, they develop preferences for sites that they are more attached to, and do their best to avoid. Now these affinities are developed through a combination of means ranging from affiliations, vocabulary, prose, and so forth.

What I would like to examine in this project is if it is possible to differentiate from several different publications with their respective perks / quirks. 

## About the Data

This dataset was obtained from Kaggle, and contains a collection of 142,570 articles from 15 different publications.

The publications within this dataset are
1. CNN
2. Breitbart
3. Vox
4. Washington Post
5. New York Post
6. National Review
7. NPR
8. Guardian
9. Talking Points Memo
10. Atlantic
11. Reuters
12. Fox News
13. Business Insider
14. Buzzfeed News
15. New York Times

## Research Question

As this is an unsupervised learning project first and foremost, the project will have 3 goals.

1. The first goal is to prepare the articles in the dataset for modelling using various Natural Language Processing (NLP) methods to re-represent the data in numbers rather than words
2. Cluster the data to determine if we can identify the articles and associate them as different groups.
3. Determine if we can predict the structure of the article based on the publisher.

## Packages

In [49]:
# Basic imports
import os
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Machine Learning packages
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2
from sklearn.preprocessing import normalize
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Clustering packages
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from scipy.spatial.distance import cdist

# Natural Language processing
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_rcv1

## Data Preview

The first matter of business is to import the articles from a local directory and merge them.

In [2]:
# Create list of files from directory
filelist = os.listdir('articles')

# Import the files
df_list = [pd.read_csv(file) for file in filelist]

#concatenate them together
articles = pd.concat(df_list)

# Preview the data
articles.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [3]:
# Print the size of the dataset
articles.shape

(142570, 10)

So we have 142,570 articles in the dataset but unfortunately, NLP is quite memory intensive, so we will have to sample the dataset unless you happen to have over 120 GB of memory on your local device. Using a 10% sample still leaves us with 140,000 articles and will be used for the duration of this project.

In [4]:
# Sample the dataset for optimal performance
articles = articles.sample(frac=0.1)

In [80]:
# Print out unique publisher names
articles.publication.unique()

array(['Buzzfeed News', 'New York Post', 'Vox', 'Washington Post',
       'Breitbart', 'NPR', 'CNN', 'Guardian', 'National Review',
       'Fox News', 'Talking Points Memo', 'Business Insider', 'Atlantic',
       'Reuters', 'New York Times'], dtype=object)

In [5]:
# Describe unique occurences for each categorical variable
articles.select_dtypes(include=['object']).nunique()

title          14251
publication       15
author          3936
date            1036
url             8476
content        14248
dtype: int64

There are also other ways to trim down the dataset before processing. We aren't particularly interested in examining the dates for this research question, but it may be of interest in another. Let's check to see how many articles each author wrote; it may not be very useful to examine authors that are only responsible for a single article, as different authors from the same publisher may choose compose their works differently.

In [6]:
# Drop variables that have no impact on the outcome
articles = articles[['title', 'publication', 'author', 'content']]

In [7]:
# View most frequently occurring authors
articles.groupby(['author']).size().sort_values(ascending=False)

author
Breitbart News                                       151
Pam Key                                              146
Associated Press                                     126
Charlie Spiering                                     115
Jerome Hudson                                         94
Joel B. Pollak                                        74
John Hayward                                          74
AWR Hawkins                                           73
Daniel Nussbaum                                       67
Alex Swoyer                                           57
Ian Hanchett                                          56
Warner Todd Huston                                    53
Post Editorial Board                                  49
David A. Graham                                       46
Merrit Kennedy                                        44
Katherine Rodriguez                                   44
Jeff Poor                                             44
NPR Staff               

Well, that partly explains how there are so many authors in this dataset. It seems as though there are over 15,000 authors, and many of them have only published one article, or have co-written multiple articles with other authors. This complicates the problem, so in order to best represent each author's writing style, let's see what happens if we simply remove all authors that only published one article as is.

## Feature Selection

In [8]:
# Drop author from the dataframe if they wrote less than 5 articles
vc = articles['author'].value_counts()
u  = [i not in set(vc[vc<=4].index) for i in articles['author']]
articles = articles[u]

In [9]:
# Reprint how many unique authors there are
articles.select_dtypes(include=['object']).nunique()

title          9354
publication      15
author          605
content        9351
dtype: int64

In [10]:
# View number of articles after feature selection
articles.shape

(9360, 4)

So after removing authors that composed fewer than 5 articles, we are left with 9k articles, or 67% of the data, and roughly 600/3900 of the authors. Now, we can create a better representation of each author since each author has at least 5 articles to evaluate from.

## Text Cleaning

Now that we've chosen which articles to use, it's time to clean them up and prepare them for feature engineering. What this section covers is the removal of annoying punctuation from the content, and reducing words to their lemmas to reduce the number of words that we are examining. Finally, we'll divide the articles into training and testing sets and separate our predictor, the words in the content, and the target, the publisher.

In [11]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [12]:
# Remove annoying punctuation from the articles
articles['content'] = articles.content.map(lambda x: text_cleaner(str(x)))
articles.head()

,title,publication,author,content
16755,People Are Laughing/Screaming Over This Sex T...,Buzzfeed News,Julia Reinstein,"’ This is the Sqweel 2, an oral sex simulator ..."
17377,One Child Has Died And Five Others Were Injur...,Buzzfeed News,Salvador Hernandez,Emergency crews on scene at Quality Inn in Nil...
42797,Woman who pushed husband out of high-rise wind...,New York Post,Natalie Musumeci,An Oklahoma woman convicted of murder for shov...
28777,Watch: Trump-Clinton final presidential debate...,Vox,Andrew Prokop,The third and at long last final general elect...
33449,"Please, can someone brief the president on the...",Washington Post,Glenn Kessler,"“When you look for a job, you can’t find it an..."


In [13]:
lemmatizer = WordNetLemmatizer()

# Reduce all text to their lemmas
for article in articles['content']:
    article = lemmatizer.lemmatize(article)

In [14]:
# Identify predictor and target variables
X = articles['content']
y = articles['publication']

# Create training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Tf-idf Vectorization

The first types of features that we are going to add are the most useful words in our dataset. Now how are we going to determine which words are deemed the most "useful"? With TF-IDF vectorizer, of course.

TF tracks the term frequency, or how often each word appears in all articles of text, while idf (or Inverse Document Frequency) is a value that places less weight on variables that occur too often and lose their predictive power. Put together, it's a tool that allows us to assign an importance value to each word in the entire dataset based on frequency in each row and throughout the database.

These are the parameters that will be used for TF-IDF
1. ll words that appear in over half of the articles will be thrown out of the dataframe
2. Only words that occur more than 5 times will be tracked
3. Only the top 150 features (words) will be kept
4. Stop words will be ignored (like, as, the)
5. Cases will be ignored
6. Shorter and longer articles will be treated equally
7. Add 1 to document frequency in case we have to divide by 0

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.5,
                             min_df=5, 
                             max_features=150, 
                             stop_words='english', 
                             lowercase=True, )
                             use_idf=True,
                             norm=u'l2',
                             smooth_idf=True
                            )

#Applying the vectorizer
X_tfidf=vectorizer.fit_transform(X)
print("Number of features: %d" % X_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=42)

#Removes all zeros from the matrix
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]

#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]

#List of features
terms = vectorizer.get_feature_names()

#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

# Normalize the dataset    
X_norm = normalize(X_train_tfidf)

# Convert from tf-idf matrix to dataframe
X_normal  = pd.DataFrame(data=X_norm.toarray())

Number of features: 150


### Phrase count with spacy

The second set of variables that we will be creating are counters of how often each publishers makes use of each part of speech, meaning adverbs, verbs, nouns, adjectives, as well as article length.

In [17]:
# Instantiating spaCy
nlp = spacy.load('en')
X_train_words = []

for row in X_train:
    # Processing each row for tokens
    row_doc = nlp(row)
    # Calculating length of each sentence
    sent_len = len(row_doc) 
    # Initializing counts of different parts of speech
    advs = 0
    verb = 0
    noun = 0
    adj = 0
    for token in row_doc:
        # Identifying each part of speech and adding to counts
        if token.pos_ == 'ADV':
            advs +=1
        elif token.pos_ == 'VERB':
            verb +=1
        elif token.pos_ == 'NOUN':
            noun +=1
        elif token.pos_ == 'ADJ':
            adj +=1
    # Creating a list of all features for each sentence
    X_train_words.append([row_doc, advs, verb, noun, adj, sent_len])

# Create dataframe with count of adverbs, verbs, nouns, and adjectives
X_count = pd.DataFrame(data=X_train_words, columns=['BOW', 'ADV', 'VERB', 'NOUN', 'ADJ', 'sent_length'])

# Change token count to token percentage
for column in X_count.columns[1:5]:
    X_count[column] = X_count[column] / X_count['sent_length']

# Normalize X_count
X_counter = normalize(X_count.drop('BOW',axis=1))
X_counter  = pd.DataFrame(data=X_counter)

In [18]:
# Combine tf-idf matrix and phrase count matrix
features = pd.concat([X_counter,X_normal], ignore_index=False, axis=1)
features.head()

,0,1,2,3,4,0,1,2,3,4,...,140,141,142,143,144,145,146,147,148,149
0,0.000029,0.000172,0.000284,0.000087,1.000000,0.0,0.000000,0.073068,0.0,0.000000,...,0.0,0.233627,0.0,0.000000,0.0,0.0,0.000000,0.141409,0.00000,0.08767
1,0.000093,0.000361,0.000467,0.000220,1.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.172890,0.00000,0.00000
2,0.000103,0.000372,0.000526,0.000263,1.000000,0.0,0.000000,0.517905,0.0,0.215882,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.190630,0.000000,0.15818,0.00000
3,0.000149,0.000810,0.000711,0.000264,0.999999,0.0,0.000000,0.054353,0.0,0.000000,...,0.0,0.000000,0.0,0.147398,0.0,0.0,0.000000,0.000000,0.00000,0.00000
4,0.000082,0.000309,0.000367,0.000192,1.000000,0.0,0.112104,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.105358,0.000000,0.00000,0.00000


And now we have our list of features. It doesn't look anything like our original dataset, now does it? 

# Clustering

Now it's finally time for some unsupervised machine learning. Each article has been binarized to 1s and 0s, and it's time to determine if we can determine if each publisher has a different method for publication.

### K-means

In [29]:
# Calulate predicted values
kmeans = KMeans(n_clusters=15, init='k-means++', random_state=42, n_init=20)
y_pred = kmeans.fit_predict(features)

pd.crosstab(y_train, y_pred)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
publication,,,,,,,,,,,,,,,
Atlantic,27,13,21,22,2,28,117,13,73,24,50,11,17,20,12
Breitbart,343,33,15,109,57,97,129,32,268,90,176,57,37,191,47
Business Insider,85,7,11,18,8,15,61,72,83,15,25,14,0,28,4
Buzzfeed News,42,17,4,22,1,6,22,19,27,2,28,10,0,6,10
CNN,144,21,14,54,3,29,69,12,90,22,81,29,1,34,25
Fox News,49,9,9,24,13,12,11,5,35,6,23,10,6,46,2
Guardian,27,11,5,10,0,7,64,12,54,8,39,18,0,7,10
NPR,39,20,92,25,6,22,85,12,48,13,51,13,11,21,14
National Review,10,10,3,8,13,31,47,1,52,20,26,1,13,23,3


In [30]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score

print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_train, y_pred)))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features, y_pred, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.02276652
Silhouette Score: 0.0677047


### Spectral Clustering

In [31]:
sc = SpectralClustering(n_clusters=15)
y_pred2 = sc.fit_predict(features)

pd.crosstab(y_train, y_pred2)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
publication,,,,,,,,,,,,,,,
Atlantic,33,30,38,15,50,14,17,11,13,56,20,26,110,16,1
Breitbart,100,108,83,29,144,30,88,30,47,232,168,399,161,9,53
Business Insider,26,21,29,67,48,6,16,0,4,50,22,71,70,7,9
Buzzfeed News,8,4,12,19,16,18,17,0,10,44,6,37,21,3,1
CNN,39,30,43,8,60,23,41,0,25,142,30,95,78,11,3
Fox News,18,7,8,4,23,8,23,6,1,51,39,44,10,7,11
Guardian,8,9,24,11,39,11,10,0,10,48,6,23,68,5,0
NPR,28,23,25,11,25,21,20,6,13,75,17,28,87,88,5
National Review,35,28,6,1,34,12,6,11,2,30,18,10,54,2,12


In [32]:
print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_train, y_pred2)))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features, y_pred2, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.02637186
Silhouette Score: 0.05386364


### Affinity Propagation

In [33]:
af = AffinityPropagation()
y_pred3 = af.fit_predict(features)

pd.crosstab(y_train, y_pred3)

col_0,0,1,2,3,4,5,6,7,8,9,...,235,236,237,238,239,240,241,242,243,244
publication,,,,,,,,,,,,,,,,,,,,,
Atlantic,4,2,0,9,1,0,3,0,0,1,...,2,1,0,2,1,0,0,1,3,0
Breitbart,0,8,0,18,8,1,3,2,9,7,...,7,1,3,3,8,1,1,4,1,2
Business Insider,0,0,0,4,1,2,0,1,0,0,...,4,0,1,2,2,0,2,0,1,1
Buzzfeed News,0,0,0,5,0,0,0,2,0,3,...,3,1,0,3,0,1,1,2,1,0
CNN,0,2,2,22,1,0,2,2,0,5,...,5,2,0,3,0,0,2,1,0,2
Fox News,0,0,0,4,2,0,2,2,0,2,...,1,0,1,1,0,1,1,0,0,0
Guardian,0,0,5,3,0,0,0,0,0,2,...,3,0,0,0,0,0,1,2,1,0
NPR,0,2,0,6,1,0,0,0,0,2,...,1,4,0,2,1,0,0,0,1,2
National Review,0,0,0,1,0,0,0,0,0,0,...,2,0,0,0,0,0,0,2,1,0


In [34]:
print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_train, y_pred3))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features, y_pred3, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.01183964
Silhouette Score: 0.00831869


# Training the Model

### Random Forest

In [44]:
rfc = ensemble.RandomForestClassifier()
rfc_train = cross_val_score(rfc, features, y_train, cv=5, n_jobs=-1)

print('Random forest classifier score: {:.5f}(+/- {:.2f})'.format(rfc_train.mean(), rfc_train.std()*2))

Rnadom forest classifier score: 0.40755(+/- 0.01)


### Logistic Regression

In [46]:
lr = LogisticRegression()
lr_train = cross_val_score(lr, features, y_train, cv=5, n_jobs=-1)

print('Logistic regression score: {:.5f}(+/- {:.2f})'.format(lr_train.mean(), lr_train.std()*2))

Logistic regression score: 0.43960(+/- 0.01)


### Gradient Boosting Classifier

In [47]:
gbc = ensemble.GradientBoostingClassifier()
gbc_train = cross_val_score(gbc, features, y_train, cv=5, n_jobs=-1)

print('Gradient boosting classifier score: {:.5f}(+/- {:.2f})'.format(gbc_train.mean(), gbc_train.std()*2))

Gradient boosting classifier score: 0.49074(+/- 0.01)


### Optimized Gradient Boosting Classifier 

In [52]:
# Parameters for gradient boosting classifier
param_grid  = {'loss':['deviance'],
               'max_features': ['sqrt'],
               'n_estimators': [400, 800],
               'max_depth': [12, 20],
               "min_samples_leaf" : [12, 20]}

# Run grid search to find ideal parameters
gbc_grid = GridSearchCV(gbc, param_grid = param_grid, n_jobs=-1)

# Initialize and fit the model.
gbc_grid.fit(features, y_train)

# Return best parameters and best score
print('Best parameters:')
print(gbc_grid.best_params_)
print('Best Score:')
print(gbc_grid.best_score_)

Best parameters:
{'loss': 'deviance', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 12, 'n_estimators': 800}
Best Score:
0.5022792022792023


# Testing the Model

In [ ]:
# Normalize Tf-idf vectors
X_test_norm = normalize(X_test_tfidf)

In [55]:
X_test_words = []

for row in X_test:
    # Processing each row for tokens
    row_doc = nlp(row)
    # Calculating length of each sentence
    sent_len = len(row_doc) 
    # Initializing counts of different parts of speech
    advs = 0
    verb = 0
    noun = 0
    adj = 0
    for token in row_doc:
        # Identifying each part of speech and adding to counts
        if token.pos_ == 'ADV':
            advs +=1
        elif token.pos_ == 'VERB':
            verb +=1
        elif token.pos_ == 'NOUN':
            noun +=1
        elif token.pos_ == 'ADJ':
            adj +=1
    # Creating a list of all features for each sentence
    X_test_words.append([row_doc, advs, verb, noun, adj, sent_len])
    
# Data frame for features
X_test_count = pd.DataFrame(data=X_test_words, columns=['BOW', 'ADV', 'VERB', 'NOUN', 'ADJ', 'sent_length'])

# Change token count to token percentage
for column in X_test_count.columns[1:5]:
    X_test_count[column] = X_test_count[column] / X_test_count['sent_length']

# Normalize X_count
X_test_counter = normalize(X_test_count.drop('BOW',axis=1))
X_test_counter  = pd.DataFrame(data=X_test_counter)

In [71]:
# Combining features into one data frame
X_test_norm_df = pd.DataFrame(data=X_test_norm.toarray())
features_test = pd.concat([X_test_counter, X_test_norm_df], ignore_index=False, axis=1)
features_test.head()

,0,1,2,3,4,0,1,2,3,4,...,140,141,142,143,144,145,146,147,148,149
0,0.000028,0.000195,0.000313,0.000123,1.000000,0.0,0.0,0.081184,0.210982,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.194815
1,0.000084,0.000358,0.000308,0.000160,1.000000,0.0,0.0,0.444061,0.000000,0.111061,...,0.0,0.0,0.100982,0.0,0.000000,0.000000,0.0,0.171878,0.162751,0.000000
2,0.000078,0.000337,0.000380,0.000121,1.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.254268,0.0,0.000000,0.125655,0.0,0.000000,0.000000,0.000000
3,0.000127,0.000375,0.000487,0.000185,1.000000,0.0,0.0,0.117811,0.000000,0.000000,...,0.0,0.0,0.133954,0.0,0.148515,0.000000,0.0,0.000000,0.000000,0.141354
4,0.000203,0.001014,0.001251,0.000406,0.999999,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [75]:
# Calulate predicted values
kmeans = KMeans(n_clusters=15, init='k-means++', random_state=42, n_init=20)
y_pred_test = kmeans.fit_predict(features_test)

pd.crosstab(y_test, y_pred_test)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
publication,,,,,,,,,,,,,,,
Atlantic,2,19,4,3,39,30,3,7,6,3,1,5,16,5,9
Breitbart,19,126,25,33,38,117,13,30,17,30,11,61,61,7,8
Business Insider,3,29,0,2,12,25,2,6,3,2,13,12,7,4,3
Buzzfeed News,1,11,0,7,4,9,6,4,8,1,5,5,11,2,0
CNN,1,44,0,20,22,30,10,10,1,12,2,4,19,8,6
Fox News,4,17,1,9,2,15,3,3,2,6,0,17,10,2,4
Guardian,0,18,0,8,14,9,4,5,4,3,3,0,12,6,1
NPR,4,25,4,3,21,18,9,3,6,8,3,8,9,4,28
National Review,5,10,4,3,16,21,1,7,1,10,0,6,10,2,3


In [76]:
print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_test, y_pred_test)))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features_test, y_pred_test, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.02235029
Silhouette Score: 0.07125383


In [77]:
X2_test_c = pd.DataFrame(features_test)
X2_test_c['kmeans_clust'] = y_pred_test

In [78]:
gbc_grid_scores_test = cross_val_score(gbc_grid, features_test, y_test, cv=5)
print('Test set score: {:.5f}(+/- {:.3f})'.format(gbc_grid_scores_test.mean(), gbc_grid_scores_test.std()*2))

Test set score: 0.46748(+/- 0.033)


# Conclusion

# Source

https://www.kaggle.com/snapcrack/all-the-news